In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator

In [2]:
def loadData():
    file = open('./dataset/vietnamese_currency.data', 'rb')
    (images, labels) = pickle.load(file)
    file.close()
    print(images.shape)
    print(labels.shape)
    return images, labels

In [3]:
X,y = loadData()
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=100)

(2712, 128, 128, 3)
(2712, 12)


In [6]:
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.1,
        rescale=1./255,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        brightness_range=[0.2,1.5], fill_mode="nearest")

In [8]:
# Create model using Sequential API
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(128,128,3)))
model.add(layers.Conv2D(256, (3,3), padding="same", activation='relu', kernel_initializer='he_uniform'))
model.add(layers.MaxPool2D((2,2), strides=(2,2)))

model.add(layers.Conv2D(128, (3,3), padding="same", activation='relu', kernel_initializer='he_uniform'))
model.add(layers.MaxPool2D((2,2), strides=(2,2)))

model.add(layers.Conv2D(64, (3,3), padding="same", activation='relu', kernel_initializer='he_uniform'))
model.add(layers.MaxPool2D((2,2), strides=(2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(12))
model.add(layers.Softmax())

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
# Training
history = model.fit(aug.flow(X_train, y_train, batch_size=128), epochs=45,validation_data=aug.flow(X_test, y_test, batch_size=128), verbose=1)
model.save("model.h5")

Epoch 1/45
15/17 [=========================>....] - ETA: 24s - loss: 4.0741 - accuracy: 0.0998

KeyboardInterrupt: 